#### 1. Setup and parameters
requires: 2 c-arm videos per run folder, "training" folder with 2D distorted marker exports in experiment root

In [23]:
import warnings
warnings.simplefilter('ignore')
import deeplabcut
import pandas as pd
import ruamel.yaml
import sys
import os
import tensorflow as tf
import numpy as np
import re

wd=r"C:\Users\Phil\Development\DeepLabCut\dev"
possum_megavids=[r"C:\Users\Phil\Development\DeepLabCut\dev\possums_mega\separate\possums_cam1_concat_skip4.mp4",
                 r"C:\Users\Phil\Development\DeepLabCut\dev\possums_mega\separate\possums_cam2_concat_skip4.mp4"]
experimenter='Phil'
markerlist = ['Body_ds1_crn',
              'Body_ds2_int',
              'Body_ds3_cdl',
              'Body_vn1_crn',
              'Body_vn2_int',
              'Body_vn3_cdl',
              'Body_acc',
              'Scapula_acr',
              'Scapula_spi',
              'Scapula_vtb',
              'Scapula_acc',
              'Humerus_dpc',
              'Humerus_ent',
              'Humerus_ect',
              'Humerus_acc',
              'Ulna_olc',
              'Ulna_int',
              'Ulna_dst',
              'Ulna_acc',
              'Radius_prx',
              'Radius_int',
              'Radius_dst',
              'Radius_acc',
              'Teres_maj_prx',
              'Teres_maj_dst',
              'Biceps_prx',
              'Biceps_dst']
dotsize = 5
corner2move2 = 512

#### 2. Rename c-arm videos with trial name for easier id

#### 3. Create new project (1 per animal)

In [24]:
task='possums_mega_separate'
path_config_file=deeplabcut.create_new_project(task,experimenter,possum_megavids, working_directory=wd, videotype='.mp4', copy_videos=False) 

Created "C:\Users\Phil\Development\DeepLabCut\dev\possums_mega_separate-Phil-2020-04-12\videos"
Created "C:\Users\Phil\Development\DeepLabCut\dev\possums_mega_separate-Phil-2020-04-12\labeled-data"
Created "C:\Users\Phil\Development\DeepLabCut\dev\possums_mega_separate-Phil-2020-04-12\training-datasets"
Created "C:\Users\Phil\Development\DeepLabCut\dev\possums_mega_separate-Phil-2020-04-12\dlc-models"
Creating the symbolic link of the video
Created the symlink of C:\Users\Phil\Development\DeepLabCut\dev\possums_mega\separate\possums_cam1_concat_skip4.mp4 to C:\Users\Phil\Development\DeepLabCut\dev\possums_mega_separate-Phil-2020-04-12\videos\possums_cam1_concat_skip4.mp4
Created the symlink of C:\Users\Phil\Development\DeepLabCut\dev\possums_mega\separate\possums_cam2_concat_skip4.mp4 to C:\Users\Phil\Development\DeepLabCut\dev\possums_mega_separate-Phil-2020-04-12\videos\possums_cam2_concat_skip4.mp4
C:\Users\Phil\Development\DeepLabCut\dev\possums_mega_separate-Phil-2020-04-12\videos

#### 4. Overwrite default bodyparts with XROMM marker list

In [25]:
%%capture
config = ruamel.yaml.load(open(path_config_file))
config['bodyparts']=markerlist
config['dotsize']=dotsize
config['corner2move2']=[corner2move2,corner2move2]
ruamel.yaml.round_trip_dump(config, sys.stdout)
with open(path_config_file, 'w') as fp:
    ruamel.yaml.round_trip_dump(config, fp)
    fp.close()

#### 5. Extract frames from vid_list for training set

In [26]:
%matplotlib inline
deeplabcut.extract_frames(path_config_file, userfeedback=False) 

22it [00:00, 215.80it/s]

Config file read successfully.
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 1824.43  seconds.
Extracting and downsampling... 12771  frames from the video.


12771it [00:53, 238.46it/s]


Kmeans clustering ... (this might take a while)


22it [00:00, 213.71it/s]

Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 1824.86  seconds.
Extracting and downsampling... 12774  frames from the video.


12774it [00:53, 237.46it/s]


Kmeans clustering ... (this might take a while)

Frames were selected.
You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


In [27]:
deeplabcut.label_frames(path_config_file)

#### 6. Convert XMALab exports to DeepLabCut format
No spaces in marker names, otherwise 2D export fails (more header columns than data)


In [ ]:
def dlc2xmalab(run):
    ## import DLC predicted labels
    df = pd.read_hdf(run, , key='df_with_missing', mode='w')
    

In [28]:
run_names = np.unique(np.array([vid.split("\\")[-2] for vid in vid_list]))
labeled_data_path=os.path.join(path_config_file.split('config.yaml')[0],"labeled-data\\")

def hflip(x_old, width):
    x_flipped = width - 1 - x_old
    return x_flipped

def xmalab2dlc(run,csv_directory,labeled_data_path, width=1024, h_flip=False):  
    ## import XMAlab 2D exports
    df = pd.read_csv(csv_directory+"\\"+run+".csv", sep=',', header=0, dtype='float', na_values=' NaN ')
    ## coerce data into DeepLabCut hierarchical format
    df['frame_index']=df.index
    df['scorer']=experimenter
    df = df.melt(id_vars=['frame_index','scorer'])
    new = df['variable'].str.rsplit("_",n=2,expand=True)
    df['variable'],df['cam'],df['coords'] = new[0], new[1], new[2]
    df=df.rename(columns={'variable':'bodyparts'})
    df['coords']=df['coords'].str.rstrip(" ").str.lower()
    if h_flip == True:
        df['value'][df['coords']=='x']= df['value'][df['coords']=='x'].apply(lambda x:width-1-x)
    df['bodyparts']=df['bodyparts'].str.lstrip(" ").astype("category")
    df['bodyparts'].cat.set_categories(markerlist,inplace=True)
    df['frame_index'] = ['labeled-data\\' + run+"Cam"+x[-1] + '\\img' + (f"{y:03d}") + '.png' for x, y in zip(df['cam'], df['frame_index'])]
    newdf = df.pivot_table(columns=['scorer', 'bodyparts', 'coords'],index='frame_index',values='value',aggfunc='first',dropna=False)
    newdf.index.name=None
    ## go into frame folders and get frame index ##
    extracted_frames = []
    for root, dirs, files in os.walk(labeled_data_path):
        for name in files:
            if name.endswith(".png") and run in root:
                camera_id = root.split(' ')[-1][-1]
                frame_no = int(name.split('.')[0].replace('img',''))
                new_name = 'labeled-data\\'+run+"Cam"+camera_id+'\\img' + (f"{frame_no:03d}") + '.png'
                extracted_frames.append(new_name)

    ## filter by list of extracted frames
    df_extracted = newdf.filter(items=pd.Index(extracted_frames),axis=0)

    ## split new df into cams 1 and 2
    df1 = df_extracted.filter(like=run+"Cam"+"1",axis=0)
    df2 = df_extracted.filter(like=run+"Cam"+"2",axis=0)

    ## split new df into cams 1 and 2, export as h5 and csv
    for x in [1,2]:
        cam_name = run+"Cam"+str(x)
        dfx = df_extracted.filter(like=cam_name,axis=0)
        data_name = labeled_data_path+cam_name+"\\CollectedData_"+experimenter+".h5"
        dfx.to_hdf(data_name, 'df_with_missing', format='table', mode='w')
        dfx.to_csv(data_name.split('.h5')[0]+'.csv')
        print("saved "+str(data_name))

        
for run in run_names:
    xmalab2dlc(run,csv_directory,labeled_data_path, h_flip=True)

saved E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run1retpro1_90-35-4msCam1\CollectedData_Phil.h5
saved E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run1retpro1_90-35-4msCam2\CollectedData_Phil.h5
saved E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run3circles_90-35-4msCam1\CollectedData_Phil.h5
saved E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run3circles_90-35-4msCam2\CollectedData_Phil.h5
saved E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run4addabd_90-35-4msCam1\CollectedData_Phil.h5
saved E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run4addabd_90-35-4msCam2\CollectedData_Phil.h5
saved E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run5flexextLAR_90-35-4msCam1\CollectedData_Phil.h5
saved E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data

#### 7. Check substituted labels

In [29]:
##UNCOMMENT 2 LINES TO MANUALLY LABEL WITH GUI
# %gui wx
# deeplabcut.label_frames(path_config_file)
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Phil.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run1retpro1_90-35-4msCam1_labeled.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run1retpro1_90-35-4msCam2_labeled.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run3circles_90-35-4msCam1_labeled.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run3circles_90-35-4msCam2_labeled.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run4addabd_90-35-4msCam1_labeled.
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-10-02\labeled-data\run4addabd_90-35-4msCam2_labeled.
They are stored in the following folder: E:\Use

#### 8. Create training set

In [30]:
deeplabcut.create_training_dataset(path_config_file)

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


#### 9. Start training

In [ ]:
deeplabcut.train_network(path_config_file, displayiters=50,saveiters=10000, maxiters=300000)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26]],
 'all_joints_names': ['Body_ds1_crn',
                      'Body_ds2_int',
                      'Body_ds3_cdl',
                      'Body_vn1_crn',
                      'Body_vn2_int',
                      'Body_vn3_cdl',
                      'Body_acc',
                      'Scapula_acr',
                      'Scapula_spi',
                      'Scapula_vtb',
                      'Scapula_acc

Starting with standard pose-dataset loader.
INFO:tensorflow:Restoring parameters from C:\Users\LabAdmin\.conda\envs\dlc-windowsGPU\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt
Max_iters overwritten as 300000
Display_iters overwritten as 50
Save_iters overwritten as 10000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'E:\\Users\\Phil\\DeepLabCut\\dev\\1_3_4_5_unenhanced-Phil-2019-10-02\\dlc-models\\iteration-0\\1_3_4_5_unenhancedOct2-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror':

iteration: 50 loss: 0.1459 lr: 0.005
iteration: 100 loss: 0.0254 lr: 0.005
iteration: 150 loss: 0.0266 lr: 0.005
iteration: 200 loss: 0.0225 lr: 0.005
iteration: 250 loss: 0.0225 lr: 0.005
iteration: 300 loss: 0.0235 lr: 0.005
iteration: 350 loss: 0.0223 lr: 0.005
iteration: 400 loss: 0.0195 lr: 0.005
iteration: 450 loss: 0.0229 lr: 0.005
iteration: 500 loss: 0.0202 lr: 0.005
iteration: 550 loss: 0.0208 lr: 0.005
iteration: 600 loss: 0.0196 lr: 0.005
iteration: 650 loss: 0.0178 lr: 0.005
iteration: 700 loss: 0.0189 lr: 0.005
iteration: 750 loss: 0.0183 lr: 0.005
iteration: 800 loss: 0.0190 lr: 0.005
iteration: 850 loss: 0.0166 lr: 0.005
iteration: 900 loss: 0.0158 lr: 0.005
iteration: 950 loss: 0.0169 lr: 0.005
iteration: 1000 loss: 0.0170 lr: 0.005
iteration: 1050 loss: 0.0176 lr: 0.005
iteration: 1100 loss: 0.0140 lr: 0.005
iteration: 1150 loss: 0.0152 lr: 0.005
iteration: 1200 loss: 0.0152 lr: 0.005
iteration: 1250 loss: 0.0150 lr: 0.005
iteration: 1300 loss: 0.0136 lr: 0.005
iterat

In [15]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26]],
 'all_joints_names': ['Body_ds1_crn',
                      'Body_ds2_int',
                      'Body_ds3_cdl',
                      'Body_vn1_crn',
                      'Body_vn2_int',
                      'Body_vn3_cdl',
                      'Body_acc',
                      'Scapula_acr',
                      'Scapula_spi',
                      'Scapula_vtb',
                      'Scapula_acc

Running  DeepCut_resnet50_1_3_4_5_unenhancedSep30shuffle1_680000  with # of trainingiterations: 680000
INFO:tensorflow:Restoring parameters from E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models\iteration-0\1_3_4_5_unenhancedSep30-trainset95shuffle1\train\snapshot-680000
Analyzing data...


160it [00:12, 13.12it/s]


Done and results stored for snapshot:  snapshot-680000
Results for 680000  training iterations: 95 1 train error: 1.7 pixels. Test error: 8.85  pixels.
With pcutoff of 0.1  train error: 1.7 pixels. Test error: 4.27 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


In [4]:
def listAllDataVideos(directory,skip=""):
    vid_list=[]
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.endswith(".avi") and not re.search(skip,root):
                vid_list.append(os.path.join(root,name))
            else:
                continue
    return vid_list
                
vids_to_analyze = listAllDataVideos(vid_directory,"cals")

In [23]:
deeplabcut.analyze_videos(path_config_file,vids_to_analyze, videotype='.avi')

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26]],
 'all_joints_names': ['Body_ds1_crn',
                      'Body_ds2_int',
                      'Body_ds3_cdl',
                      'Body_vn1_crn',
                      'Body_vn2_int',
                      'Body_vn3_cdl',
                      'Body_acc',
                      'Scapula_acr',
                      'Scapula_spi',
                      'Scapula_vtb',
                      'Scapula_acc

Using snapshot-680000 for model E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models\iteration-0\1_3_4_5_unenhancedSep30-trainset95shuffle1
num_outputs =  1
INFO:tensorflow:Restoring parameters from E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models\iteration-0\1_3_4_5_unenhancedSep30-trainset95shuffle1\train\snapshot-680000
Starting to analyze %  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run10flexextLAR_95-3-4ms\run10flexextLAR_95-3-4msCam1.avi
Loading  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run10flexextLAR_95-3-4ms\run10flexextLAR_95-3-4msCam1.avi
Duration of video [s]:  26.67 , recorded with  30.0 fps!
Overall # of frames:  800  found with (before cropping) frame dimensions:  1024 1024
Starting to extract posture


810it [00:49, 16.52it/s]                                                                                               

Detected frames:  800


810it [00:49, 16.47it/s]


...

810it [00:46, 16.35it/s]                                                                                               

Detected frames:  800


810it [00:46, 17.47it/s]


Saving results in Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run9addabd_95-3-4ms...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


'DeepCut_resnet50_1_3_4_5_unenhancedSep30shuffle1_680000'

In [5]:
deeplabcut.create_labeled_video(path_config_file,vids_to_analyze)

Starting %  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run10flexextLAR_95-3-4ms ['Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run2retpro2_90-35-4ms\\run2retpro2_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.b

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:12<00:00, 63.12it/s]


Starting %  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run10flexextLAR_95-3-4ms ['Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run2retpro2_90-35-4ms\\run2retpro2_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.b

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:13<00:00, 58.94it/s]


Starting %  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run1retpro1_90-35-4ms ['Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run2retpro2_90-35-4ms\\run2retpro2_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.bice

...

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:13<00:00, 60.05it/s]


Starting %  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run9addabd_95-3-4ms ['Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run2retpro2_90-35-4ms\\run2retpro2_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:13<00:00, 59.48it/s]


Starting %  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run9addabd_95-3-4ms ['Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run10flexextLAR_95-3-4ms\\run10flexextLAR_95-3-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run1retpro1_90-35-4ms\\run1retpro1_90-35-4msCam2.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\\run2retpro2_90-35-4ms\\run2retpro2_90-35-4msCam1.avi', 'Z:\\lab\\NSF forelimb project\\Phil_lab\\C-arm\\Ex\\23Apr18.LaiRegnault.SEP85.LS.biceps

100%|████████████████████████████████████████████████████████████████████████████████| 800/800 [00:13<00:00, 59.11it/s]


## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [6]:
deeplabcut.extract_outlier_frames?

Signature: deeplabcut.extract_outlier_frames(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, outlieralgorithm='jump', comparisonbodyparts='all', epsilon=20, p_bound=0.01, ARdegree=3, MAdegree=1, alpha=0.01, extractionalgorithm='kmeans', automatic=False, cluster_resizewidth=30, cluster_color=False, opencv=True, savelabeled=True, destfolder=None)
Docstring:
   Extracts the outlier frames in case, the predictions are not correct for a certain video from the cropped video running from
   start to stop as defined in config.yaml.

   Another crucial parameter in config.yaml is how many frames to extract 'numframes2extract'.

   Parameter
   ----------
   config : string
       Full path of the config.yaml file as a string.

   videos : list
       A list of strings containing the full paths to videos for analysis or a path to the directory, where all the videos with same extension are stored.

   videotype: string, optional
       Checks for the extension of the video in case

In [9]:
deeplabcut.extract_outlier_frames(path_config_file,vid_list) #pass a specific video

network parameters: DeepCut_resnet50_1_3_4_5_unenhancedSep30shuffle1_680000
Method  jump  found  279  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.


yes/no yes


Frames from video run1retpro1_90-35-4msCam1  already extracted (more will be added)!
Loading video...
Duration of video [s]:  26.666639999999997 , recorded @  30.00003000003 fps!
Overall # of frames:  800 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 26.67  seconds.
Extracting and downsampling... 279  frames from the video.


279it [00:10, 26.08it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [77, 501, 579, 314, 482, 28, 85, 534, 612, 552, 767, 564, 108, 733, 478, 469, 357, 548, 653, 558]
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\run1retpro1_90-35-4msCam1.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.
Method  jump  found  513  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.


...

Kmeans clustering ... (this might take a while)
Let's select frames indices: [27, 278, 9, 780, 482, 349, 567, 101, 86, 445, 729, 216, 385, 794, 545, 234, 284, 432, 710, 694]
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\run5flexextLAR_90-35-4msCam1.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.
Method  jump  found  615  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.


yes/no yes


Frames from video run5flexextLAR_90-35-4msCam2  already extracted (more will be added)!
Loading video...
Duration of video [s]:  26.666639999999997 , recorded @  30.00003000003 fps!
Overall # of frames:  800 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 26.67  seconds.
Extracting and downsampling... 615  frames from the video.


615it [00:17, 34.65it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [378, 462, 239, 423, 511, 710, 266, 343, 734, 546, 365, 340, 466, 623, 332, 702, 154, 190, 396, 407]
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\run5flexextLAR_90-35-4msCam2.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.


## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [5]:
%gui wx
deeplabcut.refine_labels(path_config_file)

Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using creat

**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [6]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

Merged data sets and updated refinement iteration to 1.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [8]:
deeplabcut.create_training_dataset(path_config_file)

E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\training-datasets\iteration-1\UnaugmentedDataSet_1_3_4_5_unenhancedSep30  already exists!
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models\iteration-1\1_3_4_5_unenhancedSep30-trainset95shuffle1  already exists!
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models\iteration-1\1_3_4_5_unenhancedSep30-trainset95shuffle1//train  already exists!
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models\iteration-1\1_3_4_5_unenhancedSep30-trainset95shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [11]:
deeplabcut.train_network(path_config_file, maxiters=200000, displayiters=100)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26]],
 'all_joints_names': ['Body_ds1_crn',
                      'Body_ds2_int',
                      'Body_ds3_cdl',
                      'Body_vn1_crn',
                      'Body_vn2_int',
                      'Body_vn3_cdl',
                      'Body_acc',
                      'Scapula_acr',
                      'Scapula_spi',
                      'Scapula_vtb',
                      'Scapula_acc

Starting with standard pose-dataset loader.
INFO:tensorflow:Restoring parameters from C:\Users\LabAdmin\.conda\envs\dlc-windowsGPU\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt
Max_iters overwritten as 200000
Display_iters overwritten as 100
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'E:\\Users\\Phil\\DeepLabCut\\dev\\1_3_4_5_unenhanced-Phil-2019-09-30\\dlc-models\\iteration-1\\1_3_4_5_unenhancedSep30-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scorem

iteration: 100 loss: 0.0861 lr: 0.005
iteration: 200 loss: 0.0263 lr: 0.005
iteration: 300 loss: 0.0250 lr: 0.005
iteration: 400 loss: 0.0234 lr: 0.005
iteration: 500 loss: 0.0217 lr: 0.005
iteration: 600 loss: 0.0244 lr: 0.005
iteration: 700 loss: 0.0220 lr: 0.005
iteration: 800 loss: 0.0218 lr: 0.005
iteration: 900 loss: 0.0207 lr: 0.005
iteration: 1000 loss: 0.0201 lr: 0.005
iteration: 1100 loss: 0.0171 lr: 0.005
iteration: 1200 loss: 0.0177 lr: 0.005
iteration: 1300 loss: 0.0163 lr: 0.005
iteration: 1400 loss: 0.0161 lr: 0.005
iteration: 1500 loss: 0.0150 lr: 0.005
iteration: 1600 loss: 0.0146 lr: 0.005
iteration: 1700 loss: 0.0141 lr: 0.005
iteration: 1800 loss: 0.0160 lr: 0.005
iteration: 1900 loss: 0.0143 lr: 0.005
iteration: 2000 loss: 0.0137 lr: 0.005
iteration: 2100 loss: 0.0138 lr: 0.005
iteration: 2200 loss: 0.0138 lr: 0.005
iteration: 2300 loss: 0.0143 lr: 0.005
iteration: 2400 loss: 0.0137 lr: 0.005
iteration: 2500 loss: 0.0133 lr: 0.005
iteration: 2600 loss: 0.0135 lr: 0

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [4]:
deeplabcut.create_labeled_video?

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [7]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)

UsageError: unrecognized arguments: #for making interactive plots.
